In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns


import warnings
from warnings import filterwarnings
filterwarnings('ignore')

In [ ]:
df = pd.read_csv('hotel_bookings.csv')
df

In [ ]:
df.shape

In [ ]:
df.isna().sum()

In [ ]:
def rem_null(df):
    df.fillna(0,inplace=True)
    print(df.isna().sum())
rem_null(df)

In [ ]:
df.columns

In [ ]:
ACB = ['adults', 'children', 'babies']
for i in ACB:
    print(f'{i} has unique value as {df[i].unique()}')

In [ ]:
pd.set_option('display.max_columns',32)

In [ ]:
filtereddata = (df['children']==0) & (df['adults']==0) & (df['babies']==0)
df[filtereddata]

In [ ]:
data = df[~filtereddata]
data

In [ ]:
df.columns

# Analysing Home country of guests

In [ ]:
countrywise_data = pd.DataFrame(data,columns=['country','No of guests']).reset_index()

In [ ]:
countrywise_data

## Folium Map

In [ ]:
import folium
from folium.plugins import HeatMap

basemap = folium.Map(zoom_start=12)
basemap.save("foliummap.html", close_file=True)

In [ ]:
basemap = folium.Map()

basemap

In [ ]:
#plotly is a advanced label data visualisation library that is extensively used for deployment label visuals

import plotly.express as px


In [ ]:
import plotly.offline as pyo
import plotly.graph_objs as go
# Set notebook mode to work in offline
pyo.init_notebook_mode()

## Pyplot Map

In [ ]:
map_cg = px.choropleth(countrywise_data,
             locations = countrywise_data['country'],
             color = countrywise_data['No of guests'],
             color_continuous_scale="ylorrd",
             hover_name = countrywise_data['country'],
             title = 'Nationality of Guests')
map_cg.write_html("plotlymap.html")
map_cg.show()

## Non colored countries are not in data

In [ ]:
attenddata = data[data['is_canceled']==0]

In [ ]:
attenddata.columns

In [ ]:
plt.figure(figsize=(15,5))
sns.boxplot(x='reserved_room_type',y='adr',hue='hotel',data=attenddata)
plt.title('Price of room types per night &person')
plt.xlabel('Room type')
plt.ylabel('Price(Euro)')
plt.legend()
plt.show()


### Blue = Resort hotel , Orange = City hotel

1. for a blue box top line is 75th percentile data and middle line is 50th percentile data and below line is 25th percentile data

2. The line perpendicular to out of the box has a top most line parallel to 75th percentile data that is 100th percentile data and the down most line is 0th percentile data

#similarly for orange box

3. The data points that we see above 100% line are high values or we can say outliers and datapoints below 0% line are low values or we can say outliers

#### An outlier is a data point that is noticeably different from the rest

#### Outliers are stragglers that are extremely high or extremely low values in a data set that can throw off your stats.

### We can conclude from this that the best distribution of CITY HOTEL is almost tend to G room type && RESORT HOTEL is almost tend to H room type

# Analysing Prices of Hotels across year

In [ ]:
data_rh = data[(data['hotel']=='Resort Hotel') & (data['is_canceled']==0)]
data_ch = data[(data['hotel']=='City Hotel') & (data['is_canceled']==0)]

In [ ]:
data_rh.head(20)

In [ ]:
resort_hotel = data_rh.groupby(['arrival_date_month'])['adr'].mean().reset_index()
resort_hotel

In [ ]:
city_hotel = data_ch.groupby(['arrival_date_month'])['adr'].mean().reset_index()
city_hotel

In [ ]:
final = resort_hotel.merge(city_hotel,on='arrival_date_month')
final.columns = ['month','price_resort','price_cityhotel']
final

In [ ]:
import sort_dataframeby_monthorweek as sd

In [ ]:
def sort_data(df,colname):
    return sd.Sort_Dataframeby_Month(df,colname)
final = sort_data(final,'month')

### To view open 'room-price.html' in files

In [ ]:
room_price_over = px.line(final,x='month',y=['price_resort','price_cityhotel'],title='Room price per night over the month')
room_price_over.write_html("room-price.html")
room_price_over.show()

#### We can conclude that the prices in Resort hotel in summer is pretty much High

#### City hotel is much expensive in Spring and Autumn that Resorts

#### Resort hotel costs less comparitively to City hotel excepth in Summer,but whereas City hotels are high in cost

# Analysing Demand Of hotels

In [ ]:
data_rh

In [ ]:
rush_rh = data_rh['arrival_date_month'].value_counts().reset_index()
rush_rh.columns=['Month','No of guessts in resort']
rush_rh

In [ ]:
rush_ch = data_ch['arrival_date_month'].value_counts().reset_index()
rush_ch.columns=['Month','No of guests in city hotel']
rush_ch

In [ ]:
rush = rush_ch.merge(rush_rh,on='Month')
rush.columns = ['month','guests rush in city hotel','guests rush in resort']
rush

In [ ]:
rush = sort_data(rush,'month')
rush

### To view open 'rush-graph.html' in files

In [ ]:
rush_graph = px.line(rush,x='month',y=['guests rush in resort','guests rush in city hotel'],title= 'No of guests in each month')
rush_graph.write_html("rush-graph.html")
rush_graph.show()

#### For both Resort and City hotel the Guests are fewer in Winter
#### Guests prefer City hotels in Summer
#### City hotels get more guests every month compared to Resorts
#### Peak time for both City hotels and Resorts is Summer

## How long do people stay at the hotels?

In [ ]:
filter=data['is_canceled']==0
clean_data=data[filter]

In [ ]:
clean_data.head()

In [ ]:
clean_data.columns


In [ ]:
clean_data.head()

In [ ]:
clean_data["total_nights"] = clean_data["stays_in_weekend_nights"] + clean_data["stays_in_week_nights"]

In [ ]:
stay=clean_data.groupby(['total_nights','hotel']).agg('count').reset_index()
stay=stay.iloc[:,0:3]
stay

In [ ]:
stay=stay.rename(columns={'is_canceled':'Number of stays'})
stay

In [ ]:
plt.figure(figsize=(20, 8))
sns.barplot(x = "total_nights", y = "Number of stays" , hue="hotel",
            hue_order = ["City Hotel", "Resort Hotel"], data=stay)

# CO-RELATION

In [ ]:
data.head()

In [ ]:
data.corr()

In [ ]:
corelation = data.corr()['is_canceled']
corelation

In [ ]:
corelation.abs().sort_values(ascending=False)

In [ ]:
corelation.abs().sort_values(ascending=False)[1:]

In [ ]:
data.columns

####    From this list it is apparent that lead_time, total_of_special_requests, required_car_parking_spaces, booking_changes and previous_cancellations are the 5 most important numerical features.
####    However, to predict whether or not a booking will be canceled, the number of booking changes is a possible source of leakage, because this information can change over time.
####    I will also not include days_in_waiting_list,booking changes  and arrival_date_year.

####    The most important feature to exclude is the "reservation_status":

In [ ]:
data.groupby('is_canceled')['reservation_status'].value_counts()

In [ ]:
list_not=['days_in_waiting_list','arrival_date_year']

In [ ]:
num_features=[col for col in data.columns if data[col].dtype!='O' and col not in list_not]
num_features

In [ ]:
cat_not=['arrival_date_year', 'assigned_room_type', 'booking_changes', 'reservation_status', 'country','days_in_waiting_list']

In [ ]:
cat_features=[col for col in data.columns if data[col].dtype=='O' and col not in cat_not]
cat_features

In [ ]:
data_cat=data[cat_features]

In [ ]:
data_cat.head()

In [ ]:
data_cat['reservation_status_date'] = pd.to_datetime(data_cat['reservation_status_date'])

In [ ]:
data_cat['year'] = data_cat['reservation_status_date'].dt.year
data_cat['month'] = data_cat['reservation_status_date'].dt.month
data_cat['day'] = data_cat['reservation_status_date'].dt.day

In [ ]:
data_cat.drop('reservation_status_date',axis=1,inplace=True)

In [ ]:
data_cat['cancellation'] = data['is_canceled']
data_cat.head()

# Handelling Categorical data

 ## Mean Encoding

In [ ]:
cols = data_cat.columns[0:8]
cols

In [ ]:
for col in cols:
    dict = data_cat.groupby([col])['cancellation'].mean().to_dict()
    data_cat[col] = data_cat[col].map(dict)

In [ ]:
data_cat

In [ ]:
dataframe = pd.concat([data_cat,data[num_features]],axis=1)
dataframe.drop('cancellation',axis=1,inplace = True)
dataframe

## Handelling Outliers

In [ ]:
#distribution plot

sns.distplot(dataframe['lead_time'])
plt.show()

### ->The outqueued data are Outliers

### -> So that we take the log values of 'lead_time

In [ ]:

def handle_outliers(col):
    dataframe[col] = np.log1p(dataframe[col])

In [ ]:
handle_outliers('lead_time')

In [ ]:
sns.distplot(dataframe['lead_time'])
plt.show()

In [ ]:
#adr

In [ ]:
sns.distplot(dataframe['adr'])
plt.show()

In [ ]:
handle_outliers('adr')

In [ ]:
sns.distplot(dataframe['adr'].dropna())
plt.show()
#dropna here is used for missing value